In [13]:
from alpaca.data.historical import CryptoHistoricalDataClient

import datetime
import threading
from finrl.meta.data_processors.processor_alpaca import AlpacaProcessor
import alpaca_trade_api as tradeapi
import time
import pandas as pd
import numpy as np
import torch
import gym


# No keys required for crypto data
client = CryptoHistoricalDataClient()

In [14]:
#https://paper-api.alpaca.markets/v2

#read the keys.txt
# Function to read the keys from keys.txt
def load_keys(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
        keys = {}
        for line in lines:
            if ':' in line:
                key, value = line.strip().split(':', 1)
                keys[key.strip()] = value.strip()
        return keys

# Load the keys from keys.txt
keys = load_keys('../keys.txt')

# Extract the API key and API secret
API_KEY = keys.get('keypub')
API_SECRET = keys.get('key_sec')



#API_BASE_URL = 'https://paper-api.alpaca.markets'
API_BASE_URL = 'https://api.alpaca.markets/v2/assets?asset_class=crypto'

data_url = 'wss://data.alpaca.markets'

In [15]:
from alpaca.data.requests import CryptoBarsRequest
from alpaca.data.timeframe import TimeFrame
from enum import Enum


class TimeFrameUnit(Enum):
    Minute = "Min"
    Hour = "Hour"
    Day = "Day"
    Week = "Week"
    Month = "Month"

timeframe_10min = TimeFrame(amount=10, unit=TimeFrameUnit.Minute)

crypto_list = ['AAVE', 'AVAX', 'BAT', 'BCH', 'BTC', 'CRV', 'DOGE', 'DOT', 'ETH', 'GRT', 'LINK', 'LTC', 'MKR', 'SHIB', 'SUSHI', 'UNI', 'USDC', 'USDT', 'XTZ', 'YFI']



In [16]:
def add_str_to_crypto_list(crypto_list):
    str_to_add = "/USD"
    new_crypto_list = []
    
    for elem in crypto_list:
        new_elem = elem + str_to_add
        new_crypto_list.append(new_elem)
    return new_crypto_list
    
new_crypto_list = add_str_to_crypto_list(crypto_list)

new_crypto_list
    

['AAVE/USD',
 'AVAX/USD',
 'BAT/USD',
 'BCH/USD',
 'BTC/USD',
 'CRV/USD',
 'DOGE/USD',
 'DOT/USD',
 'ETH/USD',
 'GRT/USD',
 'LINK/USD',
 'LTC/USD',
 'MKR/USD',
 'SHIB/USD',
 'SUSHI/USD',
 'UNI/USD',
 'USDC/USD',
 'USDT/USD',
 'XTZ/USD',
 'YFI/USD']

In [17]:
# Creating request object
request_params = CryptoBarsRequest(
  symbol_or_symbols=new_crypto_list,
  timeframe=timeframe_10min,
  start="2024-05-25",
  end="2024-05-26"
)

In [18]:
# Retrieve daily bars for Bitcoin in a DataFrame and printing it
btc_bars = client.get_crypto_bars(request_params)

# Convert to dataframe
btc_bars.df



open         high          low  \
symbol  timestamp                                                          
BAT/USD 2024-05-25 00:00:00+00:00     0.248005     0.248005     0.247505   
        2024-05-25 00:10:00+00:00     0.247721     0.247780     0.247262   
        2024-05-25 00:20:00+00:00     0.247950     0.248119     0.247950   
        2024-05-25 00:30:00+00:00     0.248250     0.248250     0.248087   
        2024-05-25 00:40:00+00:00     0.248044     0.248385     0.247867   
...                                        ...          ...          ...   
YFI/USD 2024-05-25 23:10:00+00:00  7295.970000  7295.970000  7295.970000   
        2024-05-25 23:20:00+00:00  7303.231000  7309.720000  7303.231000   
        2024-05-25 23:30:00+00:00  7307.209000  7314.990000  7307.209000   
        2024-05-25 23:40:00+00:00  7306.436000  7310.150000  7306.436000   
        2024-05-26 00:00:00+00:00  7302.459000  7317.634000  7301.372000   

                                         close  volume  trade_count  vwap  
symbol  timestamp                                                          
BAT/USD 2024-05-25 00:00:00+00:00     0.247630     0.0          0.0   0.0  
        2024-05-25 00:10:00+00:00     0.247663     0.0          0.0   0.0  
        2024-05-25 00:20:00+00:00     0.248119     0.0          0.0   0.0  
        2024-05-25 00:30:00+00:00     0.248087     0.0          0.0   0.0  
        2024-05-25 00:40:00+00:00     0.248385     0.0          0.0   0.0  
...                                        ...     ...          ...   ...  
YFI/USD 2024-05-25 23:10:00+00:00  7295.970000     0.0          0.0   0.0  
        2024-05-25 23:20:00+00:00  7309.720000     0.0          0.0   0.0  
        2024-05-25 23:30:00+00:00  7310.566500     0.0          0.0   0.0  
        2024-05-25 23:40:00+00:00  7310.150000     0.0          0.0   0.0  
        2024-05-26 00:00:00+00:00  7317.634000     0.0          0.0   0.0  

[2782 rows x 7 columns]

In [19]:
btc_bars.df.index

MultiIndex([('BAT/USD', '2024-05-25 00:00:00+00:00'),
            ('BAT/USD', '2024-05-25 00:10:00+00:00'),
            ('BAT/USD', '2024-05-25 00:20:00+00:00'),
            ('BAT/USD', '2024-05-25 00:30:00+00:00'),
            ('BAT/USD', '2024-05-25 00:40:00+00:00'),
            ('BAT/USD', '2024-05-25 00:50:00+00:00'),
            ('BAT/USD', '2024-05-25 01:00:00+00:00'),
            ('BAT/USD', '2024-05-25 01:10:00+00:00'),
            ('BAT/USD', '2024-05-25 01:20:00+00:00'),
            ('BAT/USD', '2024-05-25 01:30:00+00:00'),
            ...
            ('YFI/USD', '2024-05-25 22:10:00+00:00'),
            ('YFI/USD', '2024-05-25 22:20:00+00:00'),
            ('YFI/USD', '2024-05-25 22:30:00+00:00'),
            ('YFI/USD', '2024-05-25 22:40:00+00:00'),
            ('YFI/USD', '2024-05-25 23:00:00+00:00'),
            ('YFI/USD', '2024-05-25 23:10:00+00:00'),
            ('YFI/USD', '2024-05-25 23:20:00+00:00'),
            ('YFI/USD', '2024-05-25 23:30:00+00:00'),
            

In [20]:
df1 = btc_bars.df.reset_index(inplace=False)
df1.rename(columns={'symbol':'tic'}, inplace=True)
df1

,tic,timestamp,open,high,low,close,volume,trade_count,vwap
0,BAT/USD,2024-05-25 00:00:00+00:00,0.248005,0.248005,0.247505,0.247630,0.0,0.0,0.0
1,BAT/USD,2024-05-25 00:10:00+00:00,0.247721,0.247780,0.247262,0.247663,0.0,0.0,0.0
2,BAT/USD,2024-05-25 00:20:00+00:00,0.247950,0.248119,0.247950,0.248119,0.0,0.0,0.0
3,BAT/USD,2024-05-25 00:30:00+00:00,0.248250,0.248250,0.248087,0.248087,0.0,0.0,0.0
4,BAT/USD,2024-05-25 00:40:00+00:00,0.248044,0.248385,0.247867,0.248385,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
2777,YFI/USD,2024-05-25 23:10:00+00:00,7295.970000,7295.970000,7295.970000,7295.970000,0.0,0.0,0.0
2778,YFI/USD,2024-05-25 23:20:00+00:00,7303.231000,7309.720000,7303.231000,7309.720000,0.0,0.0,0.0
2779,YFI/USD,2024-05-25 23:30:00+00:00,7307.209000,7314.990000,7307.209000,7310.566500,0.0,0.0,0.0
2780,YFI/USD,2024-05-25 23:40:00+00:00,7306.436000,7310.150000,7306.436000,7310.150000,0.0,0.0,0.0


In [21]:
df1['tic'].unique()

array(['BAT/USD', 'BCH/USD', 'BTC/USD', 'CRV/USD', 'DOGE/USD', 'DOT/USD',
       'AAVE/USD', 'AVAX/USD', 'LTC/USD', 'MKR/USD', 'SHIB/USD',
       'SUSHI/USD', 'ETH/USD', 'GRT/USD', 'LINK/USD', 'UNI/USD',
       'USDC/USD', 'USDT/USD', 'XTZ/USD', 'YFI/USD'], dtype=object)

In [22]:
from finrl.meta.data_processor import DataProcessor

data = df1

data

,tic,timestamp,open,high,low,close,volume,trade_count,vwap
0,BAT/USD,2024-05-25 00:00:00+00:00,0.248005,0.248005,0.247505,0.247630,0.0,0.0,0.0
1,BAT/USD,2024-05-25 00:10:00+00:00,0.247721,0.247780,0.247262,0.247663,0.0,0.0,0.0
2,BAT/USD,2024-05-25 00:20:00+00:00,0.247950,0.248119,0.247950,0.248119,0.0,0.0,0.0
3,BAT/USD,2024-05-25 00:30:00+00:00,0.248250,0.248250,0.248087,0.248087,0.0,0.0,0.0
4,BAT/USD,2024-05-25 00:40:00+00:00,0.248044,0.248385,0.247867,0.248385,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
2777,YFI/USD,2024-05-25 23:10:00+00:00,7295.970000,7295.970000,7295.970000,7295.970000,0.0,0.0,0.0
2778,YFI/USD,2024-05-25 23:20:00+00:00,7303.231000,7309.720000,7303.231000,7309.720000,0.0,0.0,0.0
2779,YFI/USD,2024-05-25 23:30:00+00:00,7307.209000,7314.990000,7307.209000,7310.566500,0.0,0.0,0.0
2780,YFI/USD,2024-05-25 23:40:00+00:00,7306.436000,7310.150000,7306.436000,7310.150000,0.0,0.0,0.0


In [23]:
DP = DataProcessor(data_source = 'alpaca',
                 API_KEY = API_KEY, 
                 API_SECRET = API_SECRET, 
                 API_BASE_URL = API_BASE_URL
                 )
data

Alpaca successfully connected


,tic,timestamp,open,high,low,close,volume,trade_count,vwap
0,BAT/USD,2024-05-25 00:00:00+00:00,0.248005,0.248005,0.247505,0.247630,0.0,0.0,0.0
1,BAT/USD,2024-05-25 00:10:00+00:00,0.247721,0.247780,0.247262,0.247663,0.0,0.0,0.0
2,BAT/USD,2024-05-25 00:20:00+00:00,0.247950,0.248119,0.247950,0.248119,0.0,0.0,0.0
3,BAT/USD,2024-05-25 00:30:00+00:00,0.248250,0.248250,0.248087,0.248087,0.0,0.0,0.0
4,BAT/USD,2024-05-25 00:40:00+00:00,0.248044,0.248385,0.247867,0.248385,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
2777,YFI/USD,2024-05-25 23:10:00+00:00,7295.970000,7295.970000,7295.970000,7295.970000,0.0,0.0,0.0
2778,YFI/USD,2024-05-25 23:20:00+00:00,7303.231000,7309.720000,7303.231000,7309.720000,0.0,0.0,0.0
2779,YFI/USD,2024-05-25 23:30:00+00:00,7307.209000,7314.990000,7307.209000,7310.566500,0.0,0.0,0.0
2780,YFI/USD,2024-05-25 23:40:00+00:00,7306.436000,7310.150000,7306.436000,7310.150000,0.0,0.0,0.0


In [24]:
data.loc[data['tic'] == 'BTC/USD']

,tic,timestamp,open,high,low,close,volume,trade_count,vwap
250,BTC/USD,2024-05-25 00:00:00+00:00,68575.2000,68582.7400,68509.7135,68582.7400,0.001000,1.0,68575.200
251,BTC/USD,2024-05-25 00:10:00+00:00,68623.6550,68658.5800,68623.6550,68645.8455,0.000147,1.0,68629.400
252,BTC/USD,2024-05-25 00:20:00+00:00,68612.9045,68612.9045,68612.9045,68612.9045,0.000000,0.0,0.000
253,BTC/USD,2024-05-25 00:30:00+00:00,68584.2500,68620.8290,68584.2500,68620.8290,0.000000,0.0,0.000
254,BTC/USD,2024-05-25 00:40:00+00:00,68589.7470,68591.1555,68569.6800,68591.1555,0.000000,0.0,0.000
...,...,...,...,...,...,...,...,...,...
377,BTC/USD,2024-05-25 23:20:00+00:00,69318.5600,69330.1400,69318.5600,69330.1400,0.000000,0.0,0.000
378,BTC/USD,2024-05-25 23:30:00+00:00,69330.1005,69330.1005,69274.8550,69296.0200,0.000000,0.0,0.000
379,BTC/USD,2024-05-25 23:40:00+00:00,69216.2340,69247.5450,69216.2340,69247.5450,0.000015,1.0,69216.234
380,BTC/USD,2024-05-25 23:50:00+00:00,69255.2835,69291.3885,69255.2835,69291.3885,0.000000,0.0,0.000


In [25]:
data.dtypes

tic                         object
timestamp      datetime64[ns, UTC]
open                       float64
high                       float64
low                        float64
close                      float64
volume                     float64
trade_count                float64
vwap                       float64
dtype: object

In [26]:
data.tic.values

array(['BAT/USD', 'BAT/USD', 'BAT/USD', ..., 'YFI/USD', 'YFI/USD',
       'YFI/USD'], dtype=object)

In [41]:


# data = DP.clean_data(data)
# data = DP.add_technical_indicator(data, INDICATORS)
# data = DP.add_vix(data)

Data cleaning started
align start and end dates
produce full timestamp index
Start processing tickers
Missing data for ticker: AAVE/USD. The prices are all NaN. Fill with 0.
Missing data for ticker: AVAX/USD. The prices are all NaN. Fill with 0.
Missing data for ticker: BAT/USD. The prices are all NaN. Fill with 0.
Missing data for ticker: BCH/USD. The prices are all NaN. Fill with 0.
Missing data for ticker: BTC/USD. The prices are all NaN. Fill with 0.
Missing data for ticker: CRV/USD. The prices are all NaN. Fill with 0.
Missing data for ticker: DOGE/USD. The prices are all NaN. Fill with 0.
Missing data for ticker: DOT/USD. The prices are all NaN. Fill with 0.
Missing data for ticker: ETH/USD. The prices are all NaN. Fill with 0.
Missing data for ticker: GRT/USD. The prices are all NaN. Fill with 0.
Missing data for ticker: LINK/USD. The prices are all NaN. Fill with 0.
Missing data for ticker: LTC/USD. The prices are all NaN. Fill with 0.
Missing data for ticker: MKR/USD. The pric

NameError: name 'INDICATORS' is not defined

In [ ]:
# df_btc_bars = btc_bars.df
# 
# df_btc_bars.to_csv("../src/data/btc_bars.csv")